In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import scipy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.externals import joblib 
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
import scipy.cluster.hierarchy as sch
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

Just to get the general sense of cluster data counts. +ve and -ve split for nonUS companies.

In [5]:
# +ve and -ve split for non-US companies
# Negative count is higher for non-US companies  
nonus_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone/nonus_data_cluster_predictions.csv')
nonus_data['cloud_flag'].value_counts()

0.0    3222
1.0    1989
Name: cloud_flag, dtype: int64

In [6]:
nonus_data.head()

,count_unique_identifier,sum_feature_101,sum_feature_102,sum_feature_106,sum_feature_110,sum_feature_114,sum_feature_118,sum_feature_203,sum_feature_207,sum_feature_211,sum_feature_215,sum_feature_219,sum_feature_223,sum_feature_301,sum_feature_305,sum_feature_309,sum_feature_504,sum_feature_103,sum_feature_107,sum_feature_111,sum_feature_115,sum_feature_119,sum_feature_204,sum_feature_208,sum_feature_212,sum_feature_216,sum_feature_220,sum_feature_224,sum_feature_302,sum_feature_306,sum_feature_501,sum_feature_104,sum_feature_108,sum_feature_112,sum_feature_116,sum_feature_201,sum_feature_205,sum_feature_209,sum_feature_213,sum_feature_217,sum_feature_221,sum_feature_225,sum_feature_303,sum_feature_307,sum_feature_502,sum_feature_105,sum_feature_109,sum_feature_113,sum_feature_117,sum_feature_202,sum_feature_206,sum_feature_210,sum_feature_214,sum_feature_218,sum_feature_222,sum_feature_226,sum_feature_304,sum_feature_308,sum_feature_503,sum_raw_capacity_tb,min_raw_capacity_tb,max_raw_capacity_tb,avg_raw_capacity_tb,sum_usable_capacity_tb,min_usable_capacity_tb,max_usable_capacity_tb,avg_usable_capacity_tb,sum_free_capacity_tb,min_free_capacity_tb,max_free_capacity_tb,avg_free_capacity_tb,sum_used_capacity_tb,min_used_capacity_tb,max_used_capacity_tb,avg_used_capacity_tb,last_12_months_storage_spend,product_601,product_602,product_603,product_604,product_605,product_606,product_607,product_608,product_609,product_610,product_611,product_612,total_employees,total_revenue,it_budget_total,external_total,internal_total,per_feature_101,per_feature_102,per_feature_106,per_feature_110,per_feature_114,per_feature_118,per_feature_203,per_feature_207,per_feature_211,per_feature_215,per_feature_219,per_feature_223,per_feature_301,per_feature_305,per_feature_309,per_feature_504,per_feature_103,per_feature_107,per_feature_111,per_feature_115,per_feature_119,per_feature_204,per_feature_208,per_feature_212,per_feature_216,per_feature_220,per_feature_224,per_feature_302,per_feature_306,per_feature_501,per_feature_104,per_feature_108,per_feature_112,per_feature_116,per_feature_201,per_feature_205,per_feature_209,per_feature_213,per_feature_217,per_feature_221,per_feature_225,per_feature_303,per_feature_307,per_feature_502,per_feature_105,per_feature_109,per_feature_113,per_feature_117,per_feature_202,per_feature_206,per_feature_210,per_feature_214,per_feature_218,per_feature_222,per_feature_226,per_feature_304,per_feature_308,per_feature_503,feature_101,feature_102,feature_106,feature_110,feature_114,feature_118,feature_203,feature_207,feature_211,feature_215,feature_219,feature_223,feature_301,feature_305,feature_309,feature_504,feature_103,feature_107,feature_111,feature_115,feature_119,feature_204,feature_208,feature_212,feature_216,feature_220,feature_224,feature_302,feature_306,feature_501,feature_104,feature_108,feature_112,feature_116,feature_201,feature_205,feature_209,feature_213,feature_217,feature_221,feature_225,feature_303,feature_307,feature_502,feature_105,feature_109,feature_113,feature_117,feature_202,feature_206,feature_210,feature_214,feature_218,feature_222,feature_226,feature_304,feature_308,feature_503,total_100_features,total_200_features,total_300_features,total_500_features,total_features,total_features_per_identifier,total_products,capacity_utilization,revenue_per_employee,external_budget_by_total,external_vs_internal,US,DE,GB,JP,IT,FR,CA,NL,ES,AU,ind9,ind14,ind22,ind2,ind16,ind6,ind3,ind4,ind5,ind13,emp1,emp2,emp3,emp4,emp5,emp6,emp7,emp8,company_id,group,cloud_flag,cluster,distance
0,2.0,0.0,0.0,0.0,1.110223e-16,0.0,0.0,2.0,0.0,5.551115e-17,0.0,0.0,0.0,2.000000e+00,0.0,2.000000e+00,2.0,1.387779e-17,0.0,0.0,0.0,0.0,2.0,0.0,-1.110223e-16,2.0,0.0,0.0,2.000000e+00,2.0,5.551115e-17,0.0,0.0,0.0,3.469447e-18,1.0,2.0,1.0,0.0,2.220446e-16,0.0,4.336809e-19,8.881784e-16,2.0,0.0,-4.440892e-16,0.0,0.0,0.0,0.0,0.0,0.0,1.387779e-17,0.0,-4.440892e-16,0.0,0.0,2.0,2.0,146.364,32.798,50.267,40.144571,94.447,18.766,32.781,25.024214,44

Cluster performance split by cloud flag. Note that cluster 100 corrseponds to the companies that have cloud flag 0 and were not part of any of the 'cloud' clusters 

In [7]:
nonus_data['cluster'] = nonus_data['cluster'].apply(lambda x: x if x==100 else x+6)
nonus_cluster_by_cloudFlag =  pd.DataFrame(nonus_data.groupby(['cloud_flag', 'cluster']).agg(cluster_count=('cluster', 'count'),
                                                                                             median_distance = ('distance', np.median)))
nonus_cluster_by_cloudFlag

cluster_count  median_distance
cloud_flag cluster                                
0.0        6.0                331           0.5996
           7.0                143           0.8800
           8.0                191           0.6640
           9.0                  4           1.7990
           10.0                 2           2.3210
           11.0                11           1.3420
           12.0                61           0.2260
           13.0                18           0.5110
           100.0             2461              NaN
1.0        6.0                929           1.5830
           7.0                316           3.1060
           8.0                341           4.1250
           9.0                 96           5.7300
           10.0                74           6.1900
           11.0                33          10.5100
           12.0                92           9.5400
           13.0               106          11.3875
           14.0                 1         153.9000
           15.0                 1         180.5000

In [8]:
# Filtering for cloud flag = 1
cluster_flag1 = nonus_cluster_by_cloudFlag[nonus_cluster_by_cloudFlag.index.get_level_values(0) == 1]
cluster_flag1

cluster_count  median_distance
cloud_flag cluster                                
1.0        6.0                929           1.5830
           7.0                316           3.1060
           8.0                341           4.1250
           9.0                 96           5.7300
           10.0                74           6.1900
           11.0                33          10.5100
           12.0                92           9.5400
           13.0               106          11.3875
           14.0                 1         153.9000
           15.0                 1         180.5000

Clusters 6 and 7 look compact. 8,9 and 10 look a bit spread out(Class 1). 11, 12, 13 14 and 15 also have large distance within cluster

Cluster Analysis:
Try to inspect the feature distrbution across clusters - 6,7,8,9,10,11,12 and 13. We will compare this with values from cluster 100(flagged as companies who dont have a propen

sity for cloud migration) and try to extract any behavior/patterns

Secondly, we will compare the values for all positive class companies(coud_flag =1) with cluster 100 companies. 

Based on the above two, we will try to describe the comanies in the best way in terms of features 

In [0]:
# Cluster prediction for holdout data
result = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone/results.csv') 
result = result[result['cluster'] > 5]

#### Cluster performance on holdout set

In [10]:
result = result.groupby('cluster')['cloud_flag'].agg(total_count = 'count',
                                            percent_TP = lambda x: round(100*x.mean(),2))
result

,total_count,percent_TP
cluster,,
6,15,60.00
7,9,66.67
8,8,25.00
9,5,80.00
10,2,100.00
11,2,50.00
12,1,100.00
13,5,60.00


In [11]:
cluster_performance = pd.merge(result, cluster_flag1['median_distance'], how='inner', left_index=True, right_index=True )
cluster_performance.index = cluster_performance.index.droplevel()
cluster_performance

,total_count,percent_TP,median_distance
cluster,,,
6,15,60.00,1.5830
7,9,66.67,3.1060
8,8,25.00,4.1250
9,5,80.00,5.7300
10,2,100.00,6.1900
11,2,50.00,10.5100
12,1,100.00,9.5400
13,5,60.00,11.3875


Clusters 6,7,9 are performing decently well. Eventhough 13 has a TP rate of 0.6, it is a wide spreadout cluster. 10, 11 and 12 dont have much representation

In [12]:
nonus_data.head()

,count_unique_identifier,sum_feature_101,sum_feature_102,sum_feature_106,sum_feature_110,sum_feature_114,sum_feature_118,sum_feature_203,sum_feature_207,sum_feature_211,sum_feature_215,sum_feature_219,sum_feature_223,sum_feature_301,sum_feature_305,sum_feature_309,sum_feature_504,sum_feature_103,sum_feature_107,sum_feature_111,sum_feature_115,sum_feature_119,sum_feature_204,sum_feature_208,sum_feature_212,sum_feature_216,sum_feature_220,sum_feature_224,sum_feature_302,sum_feature_306,sum_feature_501,sum_feature_104,sum_feature_108,sum_feature_112,sum_feature_116,sum_feature_201,sum_feature_205,sum_feature_209,sum_feature_213,sum_feature_217,sum_feature_221,sum_feature_225,sum_feature_303,sum_feature_307,sum_feature_502,sum_feature_105,sum_feature_109,sum_feature_113,sum_feature_117,sum_feature_202,sum_feature_206,sum_feature_210,sum_feature_214,sum_feature_218,sum_feature_222,sum_feature_226,sum_feature_304,sum_feature_308,sum_feature_503,sum_raw_capacity_tb,min_raw_capacity_tb,max_raw_capacity_tb,avg_raw_capacity_tb,sum_usable_capacity_tb,min_usable_capacity_tb,max_usable_capacity_tb,avg_usable_capacity_tb,sum_free_capacity_tb,min_free_capacity_tb,max_free_capacity_tb,avg_free_capacity_tb,sum_used_capacity_tb,min_used_capacity_tb,max_used_capacity_tb,avg_used_capacity_tb,last_12_months_storage_spend,product_601,product_602,product_603,product_604,product_605,product_606,product_607,product_608,product_609,product_610,product_611,product_612,total_employees,total_revenue,it_budget_total,external_total,internal_total,per_feature_101,per_feature_102,per_feature_106,per_feature_110,per_feature_114,per_feature_118,per_feature_203,per_feature_207,per_feature_211,per_feature_215,per_feature_219,per_feature_223,per_feature_301,per_feature_305,per_feature_309,per_feature_504,per_feature_103,per_feature_107,per_feature_111,per_feature_115,per_feature_119,per_feature_204,per_feature_208,per_feature_212,per_feature_216,per_feature_220,per_feature_224,per_feature_302,per_feature_306,per_feature_501,per_feature_104,per_feature_108,per_feature_112,per_feature_116,per_feature_201,per_feature_205,per_feature_209,per_feature_213,per_feature_217,per_feature_221,per_feature_225,per_feature_303,per_feature_307,per_feature_502,per_feature_105,per_feature_109,per_feature_113,per_feature_117,per_feature_202,per_feature_206,per_feature_210,per_feature_214,per_feature_218,per_feature_222,per_feature_226,per_feature_304,per_feature_308,per_feature_503,feature_101,feature_102,feature_106,feature_110,feature_114,feature_118,feature_203,feature_207,feature_211,feature_215,feature_219,feature_223,feature_301,feature_305,feature_309,feature_504,feature_103,feature_107,feature_111,feature_115,feature_119,feature_204,feature_208,feature_212,feature_216,feature_220,feature_224,feature_302,feature_306,feature_501,feature_104,feature_108,feature_112,feature_116,feature_201,feature_205,feature_209,feature_213,feature_217,feature_221,feature_225,feature_303,feature_307,feature_502,feature_105,feature_109,feature_113,feature_117,feature_202,feature_206,feature_210,feature_214,feature_218,feature_222,feature_226,feature_304,feature_308,feature_503,total_100_features,total_200_features,total_300_features,total_500_features,total_features,total_features_per_identifier,total_products,capacity_utilization,revenue_per_employee,external_budget_by_total,external_vs_internal,US,DE,GB,JP,IT,FR,CA,NL,ES,AU,ind9,ind14,ind22,ind2,ind16,ind6,ind3,ind4,ind5,ind13,emp1,emp2,emp3,emp4,emp5,emp6,emp7,emp8,company_id,group,cloud_flag,cluster,distance
0,2.0,0.0,0.0,0.0,1.110223e-16,0.0,0.0,2.0,0.0,5.551115e-17,0.0,0.0,0.0,2.000000e+00,0.0,2.000000e+00,2.0,1.387779e-17,0.0,0.0,0.0,0.0,2.0,0.0,-1.110223e-16,2.0,0.0,0.0,2.000000e+00,2.0,5.551115e-17,0.0,0.0,0.0,3.469447e-18,1.0,2.0,1.0,0.0,2.220446e-16,0.0,4.336809e-19,8.881784e-16,2.0,0.0,-4.440892e-16,0.0,0.0,0.0,0.0,0.0,0.0,1.387779e-17,0.0,-4.440892e-16,0.0,0.0,2.0,2.0,146.364,32.798,50.267,40.144571,94.447,18.766,32.781,25.024214,44

In [0]:
# Data Frame to keep track of the column aggregate values clusterwise
# To append values for a particular column, create a bar plot of the column. 'Chart' will be updated with the values for the clusters specified
chart = pd.DataFrame(columns=['Aggregation',6,7,8,9,10,11,12,13,100,'Cloud'], index=nonus_data.columns.values)

## Functions to plot histogram and bar charts

In [14]:

cluster_list = [6,7,8,9,10,11,12,13,100]  #not including the outlier companies in clusters 14 and 15
# Define 9 cluster colors to plot
colors = ['blue','green','orange','red','grey','pink','violet','lightgreen','black']

def plot_histogram(df, column_name, cluster_list=[], cluster_split=True, size=(10,8), kde=True):
    """"
    Summary  : Function to plot the histogram of  column in the dataframe specified
    Arguments: 
        df: dataframe to be used
        column_name : Column name in 'df' for which the histogram, has to be plotted
        cluster_list : List of clusters to be included 
        cluster_split : If true, A different histogram for each cluster will be created in a single figure
                        If false, create 2 histograms :
                            1. all clusters except 100 are selected (representative of overall cloud behavior)
                            2. cluster 100 (representative of overall non-cloud behavior)
    """"
    plt.figure(figsize=size)
    if cluster_split:
        for idx, cluster in enumerate(cluster_list):
            if cluster==100:
                temp = df[(df['cluster']==cluster) & (df['cloud_flag']==0)]
            else:
                temp = df[(df['cluster']==cluster) & (df['cloud_flag']==1)]            
            sns.kdeplot(temp[column_name], label = 'Cluster '+str(cluster), shade=True, color=colors[idx])
    else:
         sns.kdeplot(df.loc[df['cloud_flag']==1, column_name], shade=True, label='Cloud')
         clust_100 = df[(df['cluster']==100) & (df['cloud_flag']==0)]   
         sns.kdeplot(clust_100[column_name], shade=True, label='Cluster 100')
    plt.xlabel(column_name)
    plt.ylabel('Density')
    plt.title('Distribution of ' + column_name)


def plot_bar(df, column_name, cluster_list=[], cluster_split=True, aggr='median'):
    """"
    Summary  : Function to create the barplot of a column in the dataframe specified
    Arguments: 
        df: dataframe to be used
        column_name : Column name in 'df' for which the bar chart has to be plotted
        cluster_list : List of clusters to be included 
        cluster_split : If true, A bar for each cluster will be created in a single figure
                        If false, create 2 bars :
                            1. all clusters except 100 are selected (representative of overall cloud behavior)
                            2. cluster 100 (representative of overall non-cloud behavior)
    """"
    if cluster_split:
        df_filter = df[df['cluster'].apply(lambda x: x in cluster_list)]
        plt.figure(figsize=(10,8))
        total = df_filter.groupby('cluster')[column_name].agg(aggr)
        total = total.reindex(total.index.astype(int))
        sns.barplot(total.index, total.values)
        plt.xlabel('Cluster')
        plt.ylabel(aggr + ' ' + column_name)
        chart.loc[column_name, total.index] = total.values

    else:
        plt.figure(figsize=(4,6))
        total_cloud = df.groupby('cloud_flag')[column_name].agg(aggr)
        total_cloud = total_cloud[total_cloud.index==1.0].values
        chart.loc[column_name, 'Cloud'] = total_cloud[0]
        total_clust100 = df[df['cluster'] == 100][column_name].apply(aggr)
        total = pd.Series([total_cloud[0], total_clust100], index=['Cloud', 'Cluster 100'])
        sns.barplot(total.index, total.values)
        plt.ylabel(aggr + ' ' + column_name)

        chart.loc[column_name, 'Aggregation'] = aggr


SyntaxError: ignored

In [0]:
df_filter = nonus_data[nonus_data['internal_total']<40000000]
plot_histogram(df_filter, 'internal_total', cluster_list)

In [0]:
plot_histogram(df_filter, 'internal_total', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'internal_total', cluster_list=cluster_list, aggr='median')

In [0]:
plot_bar(nonus_data, 'internal_total', cluster_split=False, aggr='median')

## External vs Internal

In [0]:
plot_histogram(nonus_data, 'external_vs_internal', cluster_list=cluster_list)

In [0]:
plot_histogram(nonus_data, 'external_vs_internal', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'external_vs_internal', cluster_list)

In [0]:
plot_bar(nonus_data, 'external_vs_internal', cluster_split=False)

## Total Employees

In [0]:
df_filter = nonus_data[nonus_data['total_employees'] < 5000]
plot_histogram(df_filter, 'total_employees', cluster_list=cluster_list)

In [0]:
df_filter = nonus_data[nonus_data['total_employees'] < 10000]
plot_histogram(df_filter, 'total_employees', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'total_employees', cluster_list)

In [0]:
df_filter = nonus_data[nonus_data['cluster'].apply(lambda x: x not in [9,10])]
plot_bar(df_filter, 'total_employees', cluster_list)

In [0]:
plot_bar(nonus_data, 'total_employees', cluster_split=False)

## Total 200 features

In [0]:
plot_bar(nonus_data, 'total_200_features', cluster_list=cluster_list)

In [0]:
df_filter = nonus_data[nonus_data['cluster'].apply(lambda x: x != 11)]
plot_bar(df_filter, 'total_200_features', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'total_200_features', cluster_split=False)

### Total 100 features

In [0]:
plot_bar(nonus_data, 'total_100_features', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'total_100_features', cluster_split=False)

### Total 300 features

In [0]:
plot_bar(nonus_data, 'total_300_features', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'total_300_features', cluster_split=False)

### Total 500 features

In [0]:
plot_bar(nonus_data, 'total_500_features', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'total_500_features', cluster_split=False)

## Per feature 307

In [0]:
plot_bar(nonus_data, 'per_feature_307', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'per_feature_307', cluster_split=False)

## Emp Bands

In [0]:
for emp_band in ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6', 'emp7', 'emp8']:
    plot_bar(nonus_data, emp_band, cluster_list=cluster_list, aggr='mean')

In [0]:
for emp_band in ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6', 'emp7', 'emp8']:
    plot_bar(nonus_data, emp_band, cluster_split=False, aggr='mean')

## Capacity

#### 1. Raw capacity 

In [0]:
df_filter = nonus_data[nonus_data['avg_raw_capacity_tb'] < 500]
plot_histogram(df_filter, 'avg_raw_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'avg_raw_capacity_tb', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'avg_raw_capacity_tb', cluster_list)

In [0]:
plot_bar(nonus_data, 'avg_raw_capacity_tb', cluster_split=False)

#### 2. Usable capacity

In [0]:
df_filter = nonus_data[nonus_data['avg_usable_capacity_tb'] < 100]
plot_histogram(df_filter, 'avg_usable_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'avg_usable_capacity_tb', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'avg_usable_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'avg_usable_capacity_tb', cluster_split=False)

#### 3. Free capacity

In [0]:
df_filter = nonus_data[nonus_data['avg_free_capacity_tb'] < 100]
plot_histogram(df_filter, 'avg_free_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'avg_free_capacity_tb', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'avg_free_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'avg_free_capacity_tb', cluster_split=False)

#### 4.Used capacity

In [0]:
df_filter = nonus_data[nonus_data['avg_used_capacity_tb'] < 100]
plot_histogram(df_filter, 'avg_used_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'avg_used_capacity_tb', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'avg_used_capacity_tb', cluster_list=cluster_list)

In [0]:
plot_bar(nonus_data, 'avg_used_capacity_tb', cluster_split=False)

#### 5. Storage spend

In [0]:
df_filter = nonus_data[nonus_data['last_12_months_storage_spend'] < 1000000]
plot_histogram(df_filter, 'last_12_months_storage_spend', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'last_12_months_storage_spend', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'last_12_months_storage_spend', cluster_list=cluster_list, aggr='median')

Cluster 11 looks way off from others, Lets remove 4 and inspect the rest

In [0]:
clusterList_filter = [n for n in cluster_list if n != 11]
plot_bar(nonus_data, 'last_12_months_storage_spend', cluster_list=clusterList_filter, aggr='median')

In [0]:
plot_bar(nonus_data, 'last_12_months_storage_spend', cluster_split=False, aggr='median')

#### 6. Capacity utilization

In [0]:
df_filter = nonus_data[nonus_data['capacity_utilization'] < 1000000000]
plot_histogram(df_filter, 'capacity_utilization', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'capacity_utilization', cluster_split=False)

In [0]:
plot_bar(nonus_data, 'capacity_utilization', cluster_list=cluster_list, aggr='median')

In [0]:
plot_bar(nonus_data, 'capacity_utilization', cluster_split=False)

## Products

In [0]:
prod_list = ['product_'+str(n) for n in range(601, 613)]
for prod in prod_list:
    plot_bar(nonus_data, prod, cluster_list=cluster_list, aggr='mean')

In [0]:
for prod in prod_list:
    plot_bar(nonus_data, prod, cluster_split=False, aggr='mean')

## Budget and Revenue

#### 1. IT budget total

In [0]:
plot_bar(nonus_data, 'it_budget_total', cluster_list=cluster_list, aggr='median') 

Clearly cluster 9 is different from others.

In [0]:
clusterList_filter = [n for n in cluster_list if n not in [9,10]]
plot_bar(nonus_data, 'it_budget_total', cluster_list=clusterList_filter, aggr='median')

In [0]:
plot_bar(nonus_data, 'it_budget_total', cluster_split=False, aggr='median')

In [0]:
df_filter = nonus_data[nonus_data['it_budget_total'] < 200000000]
plot_histogram(df_filter, 'it_budget_total', cluster_list=cluster_list)

In [0]:
plot_histogram(df_filter, 'it_budget_total', cluster_split=False)

#### 2. External total

In [0]:
plot_bar(nonus_data, 'external_total', cluster_list=cluster_list, aggr='median')

In [0]:
plot_bar(nonus_data, 'external_total', cluster_list=clusterList_filter, aggr='median')

In [0]:
plot_bar(nonus_data, 'external_total', cluster_split=False, aggr='median')

#### 5.Revenue per employee

In [0]:
plot_bar(nonus_data, 'revenue_per_employee', cluster_list=cluster_list, aggr='median')

In [0]:
plot_bar(nonus_data, 'revenue_per_employee', cluster_split=False, aggr='median')

#### 6. Total Revenue

In [0]:
df_filter = nonus_data[nonus_data['total_revenue'] < 2000000000]
plot_histogram(df_filter, 'total_revenue', cluster_list)

In [0]:
plot_bar(nonus_data, 'total_revenue', cluster_list=cluster_list, aggr='median')

In [0]:
plot_bar(nonus_data, 'total_revenue', cluster_list=clusterList_filter, aggr='median')

In [0]:
plot_bar(nonus_data, 'total_revenue', cluster_split=False, aggr='median')

In [0]:
# fig, ax = plt.subplots(3, 2, figsize=(20,20))
# idx = 0
# for i in range(3):
#     for j in range(2):
#         cluster = cluster_list[idx]
#         if cluster==100:
#             temp = us_data[(us_data['cluster']==cluster) & (us_data['cloud_flag']==0)]
#         else:
#             temp = us_data[(us_data['cluster']==cluster) & (us_data['cloud_flag']==1)]            
#         sns.distplot(temp['total_products'], kde=False, bins=12, ax=ax[i,j])    
#         ax[i,j].title.set_text('Cluster '+str(cluster))
#         idx+=1

## Industry


In [0]:
industry_cols = [col for col in nonus_data if col.startswith('ind')]
for ind in industry_cols:
    plot_bar(nonus_data, ind, cluster_list, aggr='mean')

In [0]:
for ind in industry_cols:
    plot_bar(nonus_data, ind, cluster_split=False, aggr='mean')

Summary of analyses conducted on nonUS companies.The below chart shows the aggregated values for the column name in the index for differet clusters - 6,7,8,9,10,11,12,13 

###### Cluster 100: Proxy for non-cloud companies
###### Cloud : The aggregation after grouping the companies in clusters 6,7,8,9,10,11,12 and13 

In [0]:
chart_analysis = chart.dropna(how='all')
chart_analysis[[6,7,8,9,10,11,12,13,100,'Cloud']] = chart_analysis[[6,7,8,9,10,11,12,13,100,'Cloud']].astype(float).round(2)
chart_analysis